<a href="https://colab.research.google.com/github/tuongpsa/OOP/blob/master/lunarlanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 1 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (3,570 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubun

In [64]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

In [65]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


In [ ]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

In [66]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self, state_size, action_size, seed=42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)

        # --- PHẦN 1: Feature Extraction (Trích xuất đặc trưng chung) ---
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)

        # --- PHẦN 2: Tách nhánh (Dueling Streams) ---
        # Nhánh Value (V): Output là 1 giá trị (scalar)
        self.value_stream = nn.Linear(64, 1)

        # Nhánh Advantage (A): Output là vector (action_size)
        self.advantage_stream = nn.Linear(64, action_size)

    def forward(self, state):
        # Đi qua các lớp đặc trưng chung
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))

        # --- Tách luồng dữ liệu ---
        val = self.value_stream(x)       # Tính V(s)
        adv = self.advantage_stream(x)   # Tính A(s, a)

        # --- Lớp kết hợp (Aggregation Layer) ---
        # Công thức: Q(s,a) = V(s) + (A(s,a) - mean(A(s,a)))
        return val + (adv - adv.mean(dim=1, keepdim=True))

In [ ]:
from collections import deque, namedtuple
import random
import numpy as np
import torch

class ReplayMemory(object):
    """
    Bộ nhớ đệm để lưu trữ kinh nghiệm (Experience Replay)
    Giúp phá vỡ sự tương quan giữa các mẫu dữ liệu liên tiếp.
    """
    def __init__(self, capacity):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.capacity = capacity
        self.memory = []  # Danh sách lưu trữ các sự kiện

    def push(self, event):
        """Lưu trữ một sự kiện (state, action, reward, next_state, done) vào bộ nhớ"""
        self.memory.append(event)
        # Nếu vượt quá dung lượng, xóa phần tử cũ nhất
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self, batch_size):
        """Lấy ngẫu nhiên một batch các sự kiện để huấn luyện"""
        experiences = random.sample(self.memory, k=batch_size)

        # Chuyển đổi dữ liệu thành các Tensor của PyTorch để đưa vào mạng
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)

        return states, next_states, actions, rewards, dones

In [ ]:
import numpy as np
import random
import torch
import torch.optim as optim
import torch.nn.functional as F

class Agent():
    def __init__(self, state_size, action_size):
        # Thiết lập thiết bị (Ưu tiên GPU nếu có)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.state_size = state_size
        self.action_size = action_size

        # --- KHỞI TẠO MẠNG (DUELING DQN) ---
        # Lưu ý: Class 'Network' ở đây chính là class Dueling bạn đã sửa ở trên
        self.local_qnetwork = Network(state_size, action_size).to(self.device)
        self.target_qnetwork = Network(state_size, action_size).to(self.device)

        # Optimizer
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)

        # Bộ nhớ (Replay Memory)
        self.memory = ReplayMemory(replay_buffer_size)

        # Biến đếm thời gian để update sau mỗi 4 bước
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        # Lưu trải nghiệm vào bộ nhớ
        self.memory.push((state, action, reward, next_state, done))

        # Cập nhật mỗi 4 bước thời gian (UPDATE_EVERY = 4)
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            # Chỉ học nếu trong bộ nhớ đã có đủ dữ liệu
            if len(self.memory.memory) > minibatch_size:
                # SỬA LỖI: Lấy mẫu đúng theo kích thước minibatch (thường là 64)
                experiences = self.memory.sample(minibatch_size)
                self.learn(experiences, discount_factor)

    def act(self, state, epsilon=0.):
        """Trả về hành động dựa trên state hiện tại (Epsilon-greedy)"""
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)

        self.local_qnetwork.eval() # Chuyển sang chế độ đánh giá (không tính gradient)
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train() # Chuyển lại chế độ train

        # Epsilon-greedy selection
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, discount_factor):
        """Cập nhật trọng số mạng dựa trên batch trải nghiệm"""
        states, next_states, actions, rewards, dones = experiences

        # ------------------- TÍNH TOÁN TARGET (Nhãn) -------------------
        # 1. Lấy giá trị Q max cho trạng thái tiếp theo từ mạng Target
        # .detach() để không lan truyền ngược gradient qua mạng target
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)

        # (MỞ RỘNG - Nâng cao): Nếu muốn dùng Double DQN thì bỏ comment 2 dòng dưới, xóa dòng trên
        # best_actions = self.local_qnetwork(next_states).max(1)[1].unsqueeze(1)
        # next_q_targets = self.target_qnetwork(next_states).detach().gather(1, best_actions)

        # 2. Tính Q targets cho trạng thái hiện tại (Công thức Bellman)
        q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))

        # ------------------- TÍNH TOÁN EXPECTED (Dự đoán) -------------------
        # Lấy giá trị Q mà mạng Local dự đoán cho hành động đã chọn
        q_expected = self.local_qnetwork(states).gather(1, actions)

        # ------------------- TÍNH LOSS VÀ BACKPROP -------------------
        loss = F.mse_loss(q_expected, q_targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- SOFT UPDATE -------------------
        # Cập nhật từ từ mạng Target theo mạng Local
        self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

    def soft_update(self, local_model, target_model, interpolation_parameter):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

In [ ]:
agent = Agent(state_size, number_actions)

In [ ]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -177.49
Episode 200	Average Score: -111.06
Episode 300	Average Score: -32.23
Episode 400	Average Score: -21.83
Episode 500	Average Score: 44.07
Episode 600	Average Score: 111.11
Episode 700	Average Score: 143.82
Episode 800	Average Score: 169.74
Episode 855	Average Score: 201.48
Environment solved in 755 episodes!	Average Score: 201.48


In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
import os
import gymnasium as gym

def show_video_of_model(agent, env_name, filename='video.mp4'):
    # Tạo môi trường
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []

    print(f"🚀 Đang ghi hình môi trường: {env_name}...")

    while not done:
        frame = env.render()
        frames.append(frame)

        # Agent chọn hành động
        action = agent.act(state)

        # Xử lý định dạng Action (Int vs Tensor)
        if hasattr(action, "item"):
            action_step = action.item()
        else:
            action_step = action

        # Thực hiện bước đi
        state, reward, terminated, truncated, _ = env.step(action_step)
        done = terminated or truncated

    env.close()
    # Lưu video
    imageio.mimsave(filename, frames, fps=30)
    print(f"✅ Đã lưu video vào file: {filename}")

def show_video(filename='video.mp4'):
    if os.path.exists(filename):
        try:
            video = io.open(filename, 'r+b').read()
            encoded = base64.b64encode(video)
            # Code HTML hiển thị video
            display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
        except Exception as e:
            print(f"Error displaying video {filename}: {e}")
    else:
        print(f"❌ Không tìm thấy file video: {filename}")

# --- PHẦN CHẠY CHÍNH (MAIN) ---

# 1. Quay video trước (Dùng LunarLander-v3 cho Colab mới)
show_video_of_model(agent, 'LunarLander-v3', filename='ket_qua_cuoi.mp4')

# 2. Phát video vừa quay
show_video(filename='ket_qua_cuoi.mp4')

🚀 Đang ghi hình môi trường: LunarLander-v3...


✅ Đã lưu video vào file: ket_qua_cuoi.mp4
